<a href="https://colab.research.google.com/github/softmurata/colab_notebooks/blob/main/utilities/efficientnet_pytorch_lightning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import urllib.request
import zipfile

url = "https://download.pytorch.org/tutorial/hymenoptera_data.zip"
save_path = "hymenoptera_data.zip"

if not os.path.exists(save_path):
    urllib.request.urlretrieve(url, save_path)

    zip = zipfile.ZipFile(save_path)
    zip.extractall()
    zip.close()

    os.remove(save_path)

In [ ]:
!pip install pytorch-lightning
!pip install timm

In [ ]:
# インポート
import glob
import random
import pickle

import numpy as np
import pandas as pd
import pytorch_lightning as pl
import timm
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

In [ ]:
# seedの固定
def fix_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

SEED = 0
fix_seed(SEED)

In [ ]:
#@title Define Dataset class
class CustomDataset(Dataset):
  def __init__(self, image_files, transform=None):
    self.image_files = image_files  # Image path list
    self.transform = transform
  def __len__(self):
    return len(self.image_files)
  def __getitem__(self, index):
    img_path = self.image_files[index]
    img = Image.open(img_path)
    if self.transform is not None:
      img_transformed = self.transform(img)
    # pathに含まれる文字を使用してラベリングを実施
    if 'ants' in img_path:
      label = 0
    else:
      label = 1
    # ToDo:
    # img_path, angle <- txt
    # return img_transformed, angle(normalized)?
    return img_transformed, label

In [ ]:
#@title Define Lightning Data Module
class CreateDataModule(pl.LightningDataModule):
  def __init__(self, train_path, val_path, test_path, img_size=224, mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), batch_size=16):
    super().__init__()
    self.train_path = train_path
    self.val_path = val_path
    self.test_path = test_path
    self.batch_size = batch_size

    self.train_transform = transforms.Compose([
        transforms.RandomResizedCrop(img_size, scale=(0.5, 1.0)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])

    self.val_test_transform = transforms.Compose([
        transforms.Resize(img_size),
        transforms.CenterCrop(img_size),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])

  # required in case of downloading data
  def prepare_data(self):
    pass

  def setup(self, stage=None):
    if stage == "fit" or stage is None:
      self.train_dataset = CustomDataset(self.train_path, self.train_transform)
      self.val_dataset = CustomDataset(self.val_path, self.val_test_transform)
    if stage == "test" or stage is None:
      self.test_dataset = CustomDataset(self.test_path, self.val_test_transform)
  def train_dataloader(self):
    return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)
  def val_dataloader(self):
    return DataLoader(self.val_dataset, batch_size=self.batch_size)
  def test_dataloader(self):
    return DataLoader(self.test_dataset, batch_size=self.batch_size)


In [ ]:
# create DataModule Instance
fix_seed(SEED)
# valフォルダはtestとして使用
test_path = [path for path in glob.glob("/cotent/hymenoptera_data/val/*/*.jpg")]

# trainフォルダの画像を7:3でtrain:validに分割
modeling_path = [path for path in glob.glob("/content/hymenoptera_data/train/*/*.jpg")]
train_path, val_path = train_test_split(modeling_path, train_size=0.7)
# インスタンスを作成
data_module = CreateDataModule(train_path,val_path,test_path)

In [ ]:
#@title timm tips
import timm
from PIL import Image
from torchvision import transforms

model = timm.create_model(model_name="efficientnet_b0", pretrained=True)
img_size=224
mean=(0.485, 0.456, 0.406)
std=(0.229, 0.224, 0.225)

transform = transforms.Compose([
        transforms.Resize(img_size),
        transforms.CenterCrop(img_size),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
])


img = Image.open("/content/anya.jpeg")
input = transform(img).unsqueeze(0)  # [batch_size, feature_size, hs, hs]

fea = model.forward_features(input)

In [ ]:
fea.view(1, -1).shape

In [ ]:
import torch.nn as nn
class AdjustHead(nn.Module):
  def __init__(self, in_features, n_outputs):
    super().__init__()
    self.fc1 = nn.Linear(in_features, 100)
    self.fc2 = nn.Linear(100, n_outputs)
  def forward(self, x):
    bsize = x.shape[0]
    x = x.view(bsize, -1)

    x = self.fc1(x)
    x = self.fc2(x)
    return x


In [ ]:
batch_size = 1
input_features = fea.view(batch_size, -1)
in_features = input_features.shape[-1]
adjust_head = AdjustHead(in_features, 1)
output = adjust_head(input_features)

In [ ]:
output

tensor([[-0.1747]], grad_fn=<AddmmBackward0>)

In [ ]:
#@title Define Image Classifier
class ImageClassifier(pl.LightningModule):
  def __init__(self, model_name, n_classes, lr=0.0001):
    super().__init__()
    self.save_hyperparameters()

    self.model = timm.create_model(model_name, pretrained=True)
    self.model.classifier = nn.Linear(self.model.classifier.in_features, n_classes)

    self.label_criterion = torch.nn.CrossEntropyLoss()
    # self.mseloss = torch.nn.MSELoss

    self.lr = lr
  
  def forward(self, imgs, labels=None):
    preds = self.model(imgs)
    loss = 0
    if labels is not None:
      loss = self.label_criterion(preds, labels)
    return loss, preds

  def training_step(self, batch, batch_idx):
    imgs, labels = batch
    loss, preds = self.forward(imgs=imgs, labels=labels)
    return {'loss': loss, 'batch_preds': preds.detach(), 'batch_labels': labels.detach()}
  
  def validation_step(self, batch, batch_idx):
    return self.training_step(batch, batch_idx)
  
  def test_step(self, batch, batch_idx):
    return self.training_step(batch, batch_idx)

  def validation_epoch_end(self, outputs, mode="val"):
    # calculate loss
    epoch_preds = torch.cat([x['batch_preds'] for x in outputs])
    epoch_labels = torch.cat([x['batch_labels'] for x in outputs])
    epoch_loss = self.label_criterion(epoch_preds, epoch_labels)

    self.log(f"{mode}_loss", epoch_loss, logger=True)

    # calculate accuracy
    num_correct = (epoch_preds.argmax(dim=1) == epoch_labels).sum().item()
    epoch_accuracy = num_correct / len(epoch_labels)
    self.log(f"{mode}_accuracy", epoch_accuracy, logger=True)

  def test_epoch_end(self, outputs):
    return self.validation_epoch_end(outputs, "test")
  
  def configure_optimizers(self):
    parameters = self.model.parameters()
    optimizer = optim.AdamW(lr=self.lr, params=parameters)
    scheduler = {'scheduler': optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.2)}
    return [optimizer], [scheduler]

    

In [ ]:
model = ImageClassifier(model_name="efficientnet_b0", n_classes=2)

In [ ]:
#@title config for trainer
# EarlyStoppingの設定
# 3epochで'val_loss'が0.05以上減少しなければ学習をストップ
early_stop_callback = EarlyStopping(
    monitor='val_loss', min_delta=0.05, patience=3, mode='min')

# モデルの保存先
# epoch数に応じて、「epoch=0.ckpt」のような形で保存
checkpoint_callback = ModelCheckpoint(
    filename='{epoch}', monitor='val_loss', mode='min', verbose=True)

# trainerの設定
trainer = pl.Trainer(max_epochs=20,
                     accelerator="cpu",
                     # gpus=1,
                     callbacks=[checkpoint_callback, early_stop_callback],
                     log_every_n_steps=10)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 訓練開始
trainer.fit(model, data_module)

In [ ]:
#@title inference
best_model = ImageClassifier.load_from_checkpoint(checkpoint_callback.best_model_path)

In [ ]:
img_size=224
mean=(0.485, 0.456, 0.406)
std=(0.229, 0.224, 0.225)

transform = transforms.Compose([
        transforms.Resize(img_size),
        transforms.CenterCrop(img_size),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
])


img = Image.open("/content/hymenoptera_data/val/ants/10308379_1b6c72e180.jpg")
input = transform(img).unsqueeze(0)  # [batch_size, feature_size, hs, hs]
result = best_model(input)

In [ ]:
result

(0, tensor([[-0.2377,  0.2725]], grad_fn=<AddmmBackward0>))

In [ ]:
#@title Age estimation
# https://blog.kikagaku.co.jp/google-colab-drive-mount
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip /content/drive/MyDrive/utkface.zip

In [ ]:
#@title preprocess dataset

In [ ]:
import pandas as pd
import os

In [ ]:
###### 定数を定義
DATA_DIR = '/content/UTKFace'
IM_WIDTH = IM_HEIGHT = 198
TRAIN_TEST_SPLIT = 0.01  # 全体の8割を訓練データ、残り2割をテストデータにする
TRAIN_VALID_SPLIT = 0.7  # 訓練データのうち3割はバリデーションデータとして使う
ID_GENDER_MAP = {0: 'male', 1: 'female'}  # IDから性別へ変換するマップ
GENDER_ID_MAP = dict((g, i) for i, g in ID_GENDER_MAP.items())  # IDと性別の逆引き辞書
ID_RACE_MAP = {0: 'white', 1: 'black', 2: 'asian', 3: 'indian', 4: 'others'}  # IDから人種へ変換するマップ
RACE_ID_MAP = dict((r, i) for i, r in ID_RACE_MAP.items())  # IDと人種の逆引き辞書


###### ファイル名から正解ラベルを取り出す関数
def parse_filepath(filepath):
    # 年齢(int)、性別(str)、人種(str) を返す
    try:
        path, filename = os.path.split(filepath)  # 相対パスからファイル名を取り出す
        filename, ext = os.path.splitext(filename)  # 拡張子を除く
        age, gender, race, _ = filename.split("_")  # _は無名変数
        return int(age), ID_GENDER_MAP[int(gender)], ID_RACE_MAP[int(race)]

    except Exception as e:  # いくつか欠損値があるので例外処理をしておく
        print(filepath)
        return None, None, None


###### 年齢、性別、人種、ファイル名からなるDataFrameを作成
files = glob.glob(os.path.join(DATA_DIR, "*.jpg"))  # 全ての画像ファイル名をfilesという変数にまとめる
attributes = list(map(parse_filepath, files))  # 上で作成した関数にファイル名を一つずつ入力

df = pd.DataFrame(attributes)
df['file'] = files
df.columns = ['age', 'gender', 'race', 'file']
df = df.dropna()  # 欠損値は3つ
df['gender_id'] = df['gender'].map(lambda gender: GENDER_ID_MAP[gender])
df['race_id'] = df['race'].map(lambda race: RACE_ID_MAP[race])

# 10歳以下、65歳以上の人の画像は比較的少ないので使わないことにする
df = df[(df['age'] > 10) & (df['age'] < 65)]
# その中での最高年齢
max_age = df['age'].max()
min_age = df['age'].min()

/content/UTKFace/39_1_20170116174525125.jpg.chip.jpg
/content/UTKFace/61_1_20170109150557335.jpg.chip.jpg
/content/UTKFace/61_1_20170109142408075.jpg.chip.jpg


In [ ]:
df["norm_age"] = (df['age'] - df['age'].min()) / (df['age'].max() - df['age'].min())
df.head()

,age,gender,race,file,gender_id,race_id,norm_age
0,35.0,female,asian,/content/UTKFace/35_1_2_20170116191844463.jpg....,1,2,0.452830
1,21.0,male,black,/content/UTKFace/21_0_1_20170120133601830.jpg....,0,1,0.188679
2,26.0,male,black,/content/UTKFace/26_0_1_20170117195747853.jpg....,0,1,0.283019
3,24.0,female,asian,/content/UTKFace/24_1_2_20170116172739067.jpg....,1,2,0.245283
5,17.0,male,white,/content/UTKFace/17_0_0_20170117134955265.jpg....,0,0,0.113208


In [ ]:
len(df['file'].tolist())

18529

In [ ]:
import glob
import random
import pickle

import numpy as np
import pandas as pd
import pytorch_lightning as pl
import timm
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

In [ ]:
class CustomDataset(Dataset):
  def __init__(self, df, bound_list, transform=None):
    self.df = df
    self.image_files = df['file'].tolist()  # Image path list
    self.age_list = df['norm_age'].tolist()
    self.min_age, self.max_age = bound_list
    self.transform = transform
  def __len__(self):
    return len(self.image_files)
  def __getitem__(self, index):
    img_path = self.image_files[index]
    img = Image.open(img_path)
    if self.transform is not None:
      img_transformed = self.transform(img)
    # pathに含まれる文字を使用してラベリングを実施
    target_age = torch.FloatTensor([self.age_list[index]])
    # ToDo:
    # img_path, angle <- txt
    # return img_transformed, angle(normalized)?
    return img_transformed, target_age

In [ ]:
class CreateDataModule(pl.LightningDataModule):
  def __init__(self, train_df, val_df, test_df, bound_list, img_size=224, mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), batch_size=16):
    super().__init__()
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df
    self.batch_size = batch_size

    self.train_transform = transforms.Compose([
        transforms.RandomResizedCrop(img_size, scale=(0.5, 1.0)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])

    self.val_test_transform = transforms.Compose([
        transforms.Resize(img_size),
        transforms.CenterCrop(img_size),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])

    self.bound_list = bound_list

  # required in case of downloading data
  def prepare_data(self):
    pass

  def setup(self, stage=None):
    if stage == "fit" or stage is None:
      self.train_dataset = CustomDataset(self.train_df, self.bound_list, self.train_transform)
      self.val_dataset = CustomDataset(self.val_df, self.bound_list, self.val_test_transform)
    if stage == "test" or stage is None:
      self.test_dataset = CustomDataset(self.test_df, self.bound_list, self.val_test_transform)
  def train_dataloader(self):
    return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)
  def val_dataloader(self):
    return DataLoader(self.val_dataset, batch_size=self.batch_size)
  def test_dataloader(self):
    return DataLoader(self.test_dataset, batch_size=self.batch_size)

In [ ]:
import random
target_df = df.copy()

train_msk = np.random.rand(len(df)) < 0.8
val_msk = np.random.rand(len(~train_msk)) < 0.5
test_msk = ~val_msk

train_df = target_df[train_msk]
val_df = target_df[val_msk]
test_df = target_df[test_msk]

bound_list = [min_age, max_age]

print(len(train_df), len(val_df), len(test_df))

# インスタンスを作成
data_module = CreateDataModule(train_df,val_df,test_df, bound_list)

In [ ]:
class AdjustHead(nn.Module):
  def __init__(self, in_features, n_outputs):
    super().__init__()
    self.fc1 = nn.Linear(in_features, 100)
    self.fc2 = nn.Linear(100, n_outputs)
  def forward(self, x):
    bsize = x.shape[0]
    x = x.view(bsize, -1)

    x = self.fc1(x)
    x = self.fc2(x)
    x = nn.Sigmoid()(x)
    return x

class AgeEstimator(pl.LightningModule):
  def __init__(self, model_name, n_outputs, lr=0.0001):
    super().__init__()
    self.save_hyperparameters()

    self.model = timm.create_model(model_name, pretrained=True)
    in_features = self.model.classifier.in_features * 7 * 7
    self.adjust_head = AdjustHead(in_features, n_outputs)

    # self.label_criterion = torch.nn.CrossEntropyLoss()
    self.criterion = torch.nn.MSELoss()

    self.lr = lr
  
  def forward(self, imgs, targets=None):
    
    ef_preds = self.model.forward_features(imgs)
    preds = self.adjust_head(ef_preds)
    loss = 0
    if targets is not None:
      loss = self.criterion(preds, targets)
    return loss, preds

  def training_step(self, batch, batch_idx):
    imgs, targets = batch
    loss, preds = self.forward(imgs=imgs, targets=targets)
    return {'loss': loss, 'batch_preds': preds.detach(), 'batch_targets': targets.detach()}
  
  def validation_step(self, batch, batch_idx):
    return self.training_step(batch, batch_idx)
  
  def test_step(self, batch, batch_idx):
    return self.training_step(batch, batch_idx)

  def validation_epoch_end(self, outputs, mode="val"):
    # calculate loss
    epoch_preds = torch.cat([x['batch_preds'] for x in outputs])
    epoch_targets = torch.cat([x['batch_targets'] for x in outputs])
    epoch_loss = self.criterion(epoch_preds, epoch_targets)

    self.log(f"{mode}_loss", epoch_loss, logger=True)

  def test_epoch_end(self, outputs):
    return self.validation_epoch_end(outputs, "test")
  
  def configure_optimizers(self):
    parameters = list(self.model.parameters()) + list(self.adjust_head.parameters())
    optimizer = optim.AdamW(lr=self.lr, params=parameters)
    scheduler = {'scheduler': optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.2)}
    return [optimizer], [scheduler]

In [ ]:
model = AgeEstimator(model_name="efficientnet_b0", n_outputs=1)

In [ ]:
# EarlyStoppingの設定
# 3epochで'val_loss'が0.05以上減少しなければ学習をストップ
early_stop_callback = EarlyStopping(
    monitor='val_loss', min_delta=0.05, patience=3, mode='min')

# モデルの保存先
# epoch数に応じて、「epoch=0.ckpt」のような形で保存
checkpoint_callback = ModelCheckpoint(
    filename='{epoch}', monitor='val_loss', mode='min', verbose=True)

# trainerの設定
trainer = pl.Trainer(max_epochs=20,
                     accelerator="cpu",
                     # gpus=1,
                     callbacks=[checkpoint_callback, early_stop_callback],
                     log_every_n_steps=10)

In [ ]:
!rm -rf /content/lightning_logs

In [ ]:
# 訓練開始
trainer.fit(model, data_module)

In [ ]:
#@title Problem tips
# Timm: https://towardsdatascience.com/getting-started-with-pytorch-image-models-timm-a-practitioners-guide-4e77b4bf9055
# dataframe split: https://stackoverflow.com/questions/24147278/how-do-i-create-test-and-train-samples-from-one-dataframe-with-pandas
# model parameters: https://tzmi.hatenablog.com/entry/2021/04/30/105227
# float tensor: https://note.nkmk.me/python-pytorch-dtype-to/